In [1]:
import os
import psycopg2
import json

En este caso se va a suponer que la DB esta creada de antemano en Postgres
### Postgres Connection

In [2]:
os.chdir("..")
print(os.getcwd())

c:\Users\valen\Desktop\Workshop001


In [3]:
with open ("credentials.json", "r", encoding="utf-8") as file:
    credentials = json.load(file)

Para acceder a la DB en este caso se debe tener un archivo llamado credentials.json que contenga las credenciales de la db a la aque se quiere hacer la conexión

In [4]:
db_host = credentials["db_host"]
db_name = credentials["db_name"]
db_user = credentials["db_user"]
db_password = credentials["db_password"]

conn = psycopg2.connect(
    host=db_host,
    dbname=db_name,
    user=db_user,
    password=db_password
)

Creación de la tabla "candidates_raw_data"

In [5]:
cur = conn.cursor()

cur.execute("""CREATE TABLE candidates_raw_data (
            id SERIAL PRIMARY KEY,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            email TEXT NOT NULL,
            application_date DATE NOT NULL,
            country TEXT NOT NULL,
            yoe INT NOT NULL, 
            seniority TEXT NOT NULL,
            technology TEXT NOT NULL,
            code_challenge_score INT NOT NULL,
            technical_interview_score INT NOT NULL);
""")

conn.commit()

Establecer el directorio de trabajo, el siguiente bloque de codigo debe correrse solo una vez para establecer eñ directorio de trabajo correcto

In [6]:
csv_file_path = os.path.join(os.getcwd(), "data", "candidates.csv")

table_name = 'candidates_raw_data'

copy_sql = f"""
           COPY candidates_raw_data (first_name, last_name, email, application_date, country, yoe, seniority, technology, code_challenge_score, technical_interview_score)
           FROM stdin 
           DELIMITER ';' 
           CSV HEADER;
           """
with open(csv_file_path, 'r') as f:
    cur.copy_expert(sql=copy_sql, file=f)

conn.commit()

conn.close()